In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Portfolio General Settings
We assume the code is run at T before market open, so it all data on and before T-1 close. Therefore, the end_date should be today - BDay(1)

In [4]:
end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
vol_target = 0.5
max_leverage = 1

# Linear Optimization (Risk-Adjusted Return)

- How to determine the capital?

Difficulty here is portfolio might use different leverage per rebalance, looking at the asset MV might not give you the total portfolio capital (e.g. 70% leverage, your stocks only worth 70% of your capital now)

we can use the market value on Futu divides by the leverage used for last periods. This will give you the total captial for the portfolio

Then we use this capital for next rebalance

We shuold strategically fix this issue by reading capital from Futu or keep track the capital ourselves

- 2024-03-23 - Leverage Ratio = 102.4376

In [7]:
portfolio = PortfolioLinearOpt(
    capital=188103.77659574468,    
    name='ETF Buy and Hold Portfolio',
    lookback_period=60,
    opt_freq=1,
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(),    
    systems=[
        TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),        
])

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize(gamma=10, hhi=0.2)
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-03-23 03:13:31,614 - TradingSubSystemSingle [META] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-22......
2024-03-23 03:13:31,620 - TradingSubSystemSingle [META] - INFO - Volatility Target = 50.0% | Price Volatility = 30.5% | Last Scale Factor = 1.63
2024-03-23 03:13:31,622 - TradingSubSystemSingle [TSLA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-22......
2024-03-23 03:13:31,627 - TradingSubSystemSingle [TSLA] - INFO - Volatility Target = 50.0% | Price Volatility = 48.8% | Last Scale Factor = 1.01
2024-03-23 03:13:31,632 - TradingSubSystemSingle [NVDA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-22......
2024-03-23 03:13:31,637 - TradingSubSystemSingle [NVDA] - INFO - Volatility Target = 50.0% | Price Volatility = 45.7% | Last Scale Factor = 1.10
2024-03-23 03:13:31,642 - TradingSubSystemSingle [AAPL] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-03-22

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Single - META (BAH1),Single - TSLA (BAH1),Single - NVDA (BAH1),Single - AAPL (BAH1),Single - EWY (BAH1),Single - DXJ (BAH2),Single - BRK-B (BAH2),Single - SPY (BAH1),Single - QQQ (BAH1)
Measure,,,,,,,,,,,,
Cumulative Return,1.700213,1.700827,1.092981,1.603649,0.516848,3.253261,0.985760,0.893432,1.861392,1.380635,1.136998,1.139257
Annualized Return,0.263764,0.263683,0.057806,0.340968,-0.174890,0.650718,0.033607,-0.020428,0.294300,0.162053,0.075712,0.090821
Annualized Volatility,0.219724,0.218538,0.187811,0.499836,0.494518,0.490307,0.283331,0.246993,0.164639,0.180964,0.188198,0.252640
Annualized Sharpe Ratio,1.040542,1.045814,0.120722,0.611871,-0.424702,1.255511,-0.005386,-0.224949,1.574155,0.701359,0.215620,0.220424
Maximum Drawdown,-0.262826,-0.270397,-0.254251,-0.682479,-0.635372,-0.531787,-0.309294,-0.406293,-0.117846,-0.265790,-0.244964,-0.348280


# Equal Weighted Portfolios

In [8]:
# portfolio = PortfolioStandard(    
#     capital=200000,
#     name='ETF Buy and Hold Portfolio',    
#     rebalance_iter=RebalancerIter('0 0 * * Fri', 2),
#     tc_model=TransactionCostFutu(),    
#     systems=[
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),       
# ])

# portfolio.set_start_date(start_date)
# portfolio.set_end_date(end_date)
# portfolio.backtest_subsystems()
# portfolio.optimize()
# portfolio.backtest()
# portfolio.rebalance()
# portfolio.performance()

# Execute the Portfolio

In [13]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-03-23 03:20:56,560 - ExecutorFutu - INFO - market: US
2024-03-23 03:20:56,561 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-03-23 03:20:56,565 | 28613 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=26, host=127.0.0.1, port=11111, user_id=18214795
2024-03-23 03:20:56,701 | 28613 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=26


2024-03-23 03:20:59,706 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $188,104
2024-03-23 03:20:59,708 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $188,104
2024-03-23 03:20:59,716 - ExecutorFutu - INFO - Execute Linear Optimized Portfolio (ETF Buy and Hold Portfolio) position based on 2024-03-22


2024-03-23 03:20:59,721 | 28613 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=27, host=127.0.0.1, port=11111, user_id=18214795


2024-03-23 03:20:59,736 - Futu - INFO - 7 Positions: US.SPY, US.QQQ, US.NVDA, US.META, US.EWY, US.DXJ, US.BRK.B


2024-03-23 03:20:59,747 | 28613 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=27


,instrument,target,current,turnover
0,META,47.0,47.0,0.0
1,TSLA,0.0,0.0,0.0
2,NVDA,58.0,58.0,0.0
3,AAPL,0.0,0.0,0.0
4,EWY,97.0,97.0,0.0
5,DXJ,406.0,406.0,0.0
6,BRK-B,87.0,87.0,0.0
7,SPY,33.0,33.0,0.0
8,QQQ,15.0,15.0,0.0


[*********************100%***********************]  9 of 9 completed

2024-03-23 03:21:00,314 - ExecutorFutu - INFO - getting last 5m prices since 2024-03-22 15:20:00 for order limit price


""
